# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [1]:
from __future__ import division
import turicreate
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = turicreate.SFrame('amazon_baby.sframe/')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
import string 
def remove_punctuation(text):
    try: # python 2.x
        text = text.translate(None, string.punctuation) 
    except: # python 3.x
        translator = text.maketrans('', '', string.punctuation)
        text = text.translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = turicreate.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products[269]['word_count']

{'our': 1.0, 'in': 1.0, 'favorite': 1.0, 'house': 1.0, 'a': 1.0}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None, verbose=False)

In [ ]:
sentiment_model

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [13]:
sentiment_model.coefficients

name,index,class,value,stderr
(intercept),None,1,0.7797362814628314,None
word_count,recommend,1,0.30524577375828277,None
word_count,disappointed,1,-2.4267924944782595,None
word_count,wise,1,-0.18170817410610382,None
word_count,love,1,0.8405057320615058,None
word_count,it,1,0.010479446044200771,None
word_count,planet,1,-0.3846729911238415,None
word_count,and,1,0.0363143727865875,None
word_count,bags,1,0.13057598098873,None
word_count,wipes,1,-0.0020751803190873475,None


In [12]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [16]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [20]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1.0, 'waste': 1.0,'wouldnt': 1.0, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [18]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [19]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [21]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[4.788907309214033, -3.0007822224625715, -8.188501360762764]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [23]:
y_predict = [1 if scores[i]>0 else -1 for i in range(len(scores))]
y_predict

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [ ]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data))

**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [25]:
import numpy as np
probability = lambda x: 1/(1+np.exp(-x))
probability(scores)

array([9.91747131e-01, 4.73905475e-02, 2.77752771e-04])

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [26]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to Turi Create:
[0.9917471313286886, 0.04739054748711907, 0.0002777527712172623]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [34]:
test_data['predict']=sentiment_model.predict(test_data, output_type='probability')

In [36]:
test_data.topk('predict')[0:20]

name,review,rating,word_count,sentiment,predict
"Britax DecathlonConvertible Car Seat, ...",I researched a fewdifferent seats to pu ...,4.0,"{'enough': 1.0, 'big':1.0, 'carrier': 1.0, ...",1,1.0
Lilly Gold Sit 'n' Stroll5 in 1 Car Seat and ...,"I just completed a two-month, cross-country ...",5.0,"{'toddler': 1.0, 'or':1.0, 'plan': 1.0, ...",1,1.0
Fisher-Price RainforestMelodies and Lights ...,My daughter wasn't ableto interact much with ...,5.0,"{'youll': 1.0, 'afford':1.0, 'explore': 1.0, ...",1,1.0
Peg Perego Aria LightWeight One Hand Fold ...,We have 3 strollers...onefor when my son was an ...,5.0,"{'collapses': 1.0,'awesome': 1.0, ...",1,1.0
"P'Kolino Silly SoftSeating in Tias, Green ...",I've purchased both theP'Kolino Little Reader ...,4.0,"{'admit': 1.0, 'though':1.0, 'clutter': 2.0, ...",1,1.0
Evenflo X Sport PlusConvenience Stroller - ...,After seeing this inParent's Magazine and ...,5.0,"{'compliments': 1.0,'getting': 1.0, 'alwa ...",1,1.0
Britax Frontier BoosterCar Seat ...,My family previouslyowned a Britax Wizard ...,5.0,"{'looks': 1.0, 'now':1.0, 'fuzzy': 1.0, ...",1,1.0
Baby Einstein Around TheWorld Discovery Center ...,I am so HAPPY I broughtthis item for my 7 mo ...,5.0,"{'go': 1.0, 'steals':1.0, 'prices': 1.0, ...",1,1.0
"Infantino Wrap and TieBaby Carrier, Black ...",I bought this carrierwhen my daughter was ...,5.0,"{'well': 1.0, 'mouth':1.0, 'sure': 1.0, ...",1,1.0
"Fisher-Price Cradle 'NSwing, My Little ...",My husband and I cannotstate enough how much we ...,5.0,"{'recommendations': 1.0,'highest': 1.0, ...",1,1.0


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [38]:
test_data.topk('predict', reverse=True)[0:20]

name,review,rating,word_count,sentiment
"Luna Lullaby Bosom BabyNursing Pillow, Choco ...","I have the boppy deluxepillow, the brestfriend ...",5.0,"{'leacho': 1.0,'pregnant': 1.0, 'whi ...",1
The First Years TrueChoice P400 Premium ...,Note: we never installedbatteries in these un ...,1.0,"{'least': 1.0, 'wont':1.0, 'features': 1.0, ...",-1
Jolly Jumper Arctic SneakA Peek Infant Car Seat ...,"I am a ""research-aholic""in general and have ...",5.0,"{'worthless': 1.0,'wasting': 1.0, ...",1
Motorola MBP36 RemoteWireless Video Baby ...,I could go on and onabout the awful ...,4.0,"{'best': 1.0,'definitely': 1.0, ...",1
VTech Communications Safe&amp; Sounds Full Color ...,"This is my second videomonitoring system, the ...",1.0,"{'thisbuyer': 1.0,'creekive': 1.0, ...",-1
Fisher-Price OceanWonders Aquarium Bouncer ...,We have not had ANY luckwith Fisher-Price ...,2.0,"{'market': 1.0, 'same':1.0, 'mentioned': 1.0, ...",-1
Levana Safe N'See DigitalVideo Baby Monitor with ...,This is the first reviewI have ever written out ...,1.0,"{'anything': 1.0, 'post':1.0, 'honest': 1.0, ...",-1
Safety 1st High-DefDigital Monitor ...,We bought this babymonitor to replace a ...,1.0,"{'looking': 1.0,'recommend': 1.0, 'id': ...",-1
Snuza Portable BabyMovement Monitor ...,I would have given theproduct 4 stars for w ...,1.0,"{'from': 1.0,'restaurant': 1.0, 'i ...",-1
Adiri BPA Free NaturalNurser Ultimate Bottle ...,I will try to write anobjective review of the ...,2.0,"{'buy': 1.0, 'refuses':1.0, 'hisher': 1.0, ...",-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [39]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predict = model.predict(data)

    # Compute the number of correctly classified examples
    correctly = len(data[predict==true_labels])

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = correctly/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [40]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [41]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [42]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [43]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [44]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [45]:
print(train_data[0]['word_count'])

{'recommend': 1.0, 'disappointed': 1.0, 'wise': 1.0, 'love': 1.0, 'it': 3.0, 'planet': 1.0, 'and': 3.0, 'bags': 1.0, 'wipes': 1.0, 'highly': 1.0, 'not': 2.0, 'early': 1.0, 'came': 1.0, 'i': 1.0, 'does': 1.0, 'my': 2.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'holder': 1.0, 'leak': 1.0, 'keps': 1.0, 'osocozy': 1.0, 'moist': 1.0}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [ ]:
print(train_data[0]['word_count_subset'])

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [ ]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)
simple_model

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [47]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [48]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552027043,0.012088854133053259
word_count_subset,disappointed,1,-2.3550925006107253,0.050414988855697916
word_count_subset,love,1,1.3654354936790372,0.03035462951090517
word_count_subset,well,1,0.5042567463979284,0.021381300630990033
word_count_subset,product,1,-0.320555492995575,0.015431132136201635
word_count_subset,loves,1,1.6772714555592918,0.04823282753835012
word_count_subset,little,1,0.5206286360250184,0.021469147566490373
word_count_subset,work,1,-0.6217000124253143,0.023033059794584827
word_count_subset,easy,1,1.1936618983284648,0.029288869202029586
word_count_subset,great,1,0.9446912694798443,0.020950992659050018


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [49]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)


+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.6772714555592918  |
| word_count_subset |   perfect    |   1   |  1.5144862670271348  |
| word_count_subset |     love     |   1   |  1.3654354936790372  |
|    (intercept)    |     None     |   1   |  1.2995449552027043  |
| word_count_subset |     easy     |   1   |  1.1936618983284648  |
| word_count_subset |    great     |   1   |  0.9446912694798443  |
| word_count_subset |    little    |   1   |  0.5206286360250184  |
| word_count_subset |     well     |   1   |  0.5042567463979284  |
| word_count_subset |     able     |   1   |  0.1914383022947509  |
| word_count_subset |     old      |   1   |  0.0853961886678159  |
| word_count_subset |     car      |   1   | 0.05883499006802042  |
| word_count_subset |     less     |   1   | -0.

dtype: str
Rows: 21
[None, 'disappointed', 'love', 'well', 'product', 'loves', 'little', 'work', 'easy', 'great', 'would', 'able', 'perfect', 'old', 'car', 'money', 'waste', 'even', 'less', 'broke', 'return']

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [61]:
for word in significant_words:
    print(word, ':', sentiment_model.coefficients[sentiment_model.coefficients['index']==word]['value'][0])


love : 0.8405057320615058
great : 0.778953288380508
easy : 0.7349826255674922
old : 0.07967490900987567
little : 0.4099316272571707
perfect : 1.0447994204048676
loves : 0.9749823125142639
well : 0.40107557492331797
able : 0.10752802191424285
car : 0.11965787650766015
broke : -1.4746151909510392
less : -0.14682829428620733
even : -0.3143769804945452
waste : -2.1011791748728705
disappointed : -2.4267924944782595
work : -0.40240742684881686
product : 0.006637209318343504
money : -0.7994560690364586
would : -0.20046817717823925
return : -1.6918331501300319


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [62]:
accuracy_sentiment = get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])
accuracy_sentiment

0.976494573364514

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [63]:
accuracy_simple = get_classification_accuracy(simple_model, train_data, train_data['sentiment'])
accuracy_simple

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [64]:
accuracy_sentiment_TEST = get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])
accuracy_sentiment_TEST

0.9221862251019919

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [65]:
accuracy_simple_TEST = get_classification_accuracy(simple_model, test_data, test_data['sentiment'])
accuracy_simple_TEST

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [66]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [67]:
num_positive/(num_negative+num_positive)

0.8407087605684476

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?